In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 15, 'lines.linewidth': 3, 'savefig.dpi': 1200})

import os, random, time, pickle, torch
import numpy as np

export_dir = 'store/models/exported'
if not os.path.exists(export_dir):
    os.makedirs(export_dir)

In [2]:
from blurnet import trainer

job = trainer.TrainJob('store')

In [3]:
task, grayscale = 'CIFAR10', True
export_pths = {}
sigmas = {
    'baseline': None,
    'blur': 1.5,
}

for label, sigma in sigmas.items():
    cond = {'model_config': {'task': task, 'grayscale': grayscale, 'sigma': sigma}}
    export_pths[label] = []
    for key, config in job.conditioned(cond):
        export_pth = os.path.join(export_dir, '{}-{}-{}-{:03d}.pickle'.format(
            task, 'GRAY' if grayscale else 'RGB',
            'baseline' if sigma is None else '{:g}'.format(sigma),
            config['train_config']['seed']
        ))
        if not os.path.join(export_pth):
            job.export(key, export_pth)
        export_pths[label].append(export_pth)
    print('{} models exported for sigma={}'.format(len(export_pths[label]), sigma))

10 models exported for sigma=None
10 models exported for sigma=1.5
